In [ ]:
"""
Interactive Dashboard for Stock Price Prediction
Built with Plotly Dash for real-time visualization
"""

import dash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import logging

# Import custom modules
from stocks import StockDataCollector
from feature_engineering import FeatureEngineer
from models import StockPredictionModels
from visualization import StockVisualizer
import config

logger = logging.getLogger(__name__)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Initialize components
visualizer = StockVisualizer()

# Layout
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.H1("Stock Price Prediction Dashboard",
                   className="text-center mb-4",
                   style={'color': '#2c3e50', 'fontWeight': 'bold'}),
            html.Hr()
        ])
    ]),

    # Control Panel
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Control Panel"),
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            html.Label("Stock Symbol"),
                            dcc.Dropdown(
                                id='stock-symbol',
                                options=[
                                    {'label': symbol, 'value': symbol}
                                    for symbol in config.DATA_CONFIG['default_symbols']
                                ],
                                value='AAPL',
                                clearable=False
                            )
                        ], width=3),

                        dbc.Col([
                            html.Label("Date Range"),
                            dcc.DatePickerRange(
                                id='date-range',
                                start_date=(datetime.now() - timedelta(days=365)).date(),
                                end_date=datetime.now().date(),
                                display_format='YYYY-MM-DD'
                            )
                        ], width=4),

                        dbc.Col([
                            html.Label("Model Selection"),
                            dcc.Dropdown(
                                id='model-selection',
                                options=[
                                    {'label': 'Linear Regression', 'value': 'linear'},
                                    {'label': 'Random Forest', 'value': 'random_forest'},
                                    {'label': 'LSTM', 'value': 'lstm'},
                                    {'label': 'Ensemble', 'value': 'ensemble'},
                                    {'label': 'All Models', 'value': 'all'}
                                ],
                                value='all',
                                clearable=False
                            )
                        ], width=3),

                        dbc.Col([
                            html.Br(),
                            dbc.Button(
                                "Update",
                                id='update-button',
                                color='primary',
                                className='w-100'
                            )
                        ], width=2)
                    ])
                ])
            ], className="mb-4")
        ])
    ]),

    # Tabs for different views
    dbc.Row([
        dbc.Col([
            dcc.Tabs(id='main-tabs', value='overview', children=[
                dcc.Tab(label='Overview', value='overview'),
                dcc.Tab(label='Predictions', value='predictions'),
                dcc.Tab(label='Technical Analysis', value='technical'),
                dcc.Tab(label='Model Performance', value='performance'),
                dcc.Tab(label='Backtesting', value='backtesting'),
                dcc.Tab(label='Settings', value='settings')
            ])
        ])
    ]),

    # Content area
    dbc.Row([
        dbc.Col([
            html.Div(id='tab-content', className='mt-4')
        ])
    ]),

    # Status bar
    dbc.Row([
        dbc.Col([
            html.Hr(),
            html.Div(id='status-bar', className='text-center text-muted')
        ])
    ]),

    # Hidden div to store data
    html.Div(id='stored-data', style={'display': 'none'}),

    # Auto-refresh
    dcc.Interval(
        id='interval-component',
        interval=config.DASHBOARD_CONFIG['auto_refresh_interval'] * 1000,  # in milliseconds
        n_intervals=0
    )

], fluid=True)


# Helper functions for generating content
def generate_overview_content(df, symbol):
    """Generate overview tab content"""

    # Calculate summary statistics
    latest_price = df['Close'].iloc[-1]
    price_change = df['Close'].iloc[-1] - df['Close'].iloc[-2]
    price_change_pct = (price_change / df['Close'].iloc[-2]) * 100

    volume = df['Volume'].iloc[-1]
    avg_volume = df['Volume'].mean()

    high_52w = df['Close'].rolling(252).max().iloc[-1] if len(df) >= 252 else df['Close'].max()
    low_52w = df['Close'].rolling(252).min().iloc[-1] if len(df) >= 252 else df['Close'].min()

    # Create cards with metrics
    cards = dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"${latest_price:.2f}", className="text-primary"),
                    html.P("Current Price", className="text-muted"),
                    html.H6(f"{price_change:+.2f} ({price_change_pct:+.2f}%)",
                           className="text-success" if price_change >= 0 else "text-danger")
                ])
            ])
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{volume:,.0f}", className="text-info"),
                    html.P("Volume", className="text-muted"),
                    html.H6(f"Avg: {avg_volume:,.0f}")
                ])
            ])
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"${high_52w:.2f}", className="text-success"),
                    html.P("52W High", className="text-muted"),
                    html.H6(f"Low: ${low_52w:.2f}")
                ])
            ])
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{df['RSI_14'].iloc[-1]:.1f}" if 'RSI_14' in df.columns else "N/A"),
                    html.P("RSI (14)", className="text-muted"),
                    html.H6("Overbought" if 'RSI_14' in df.columns and df['RSI_14'].iloc[-1] > 70 else
                           "Oversold" if 'RSI_14' in df.columns and df['RSI_14'].iloc[-1] < 30 else "Neutral")
                ])
            ])
        ], width=3)
    ], className="mb-4")

    # Create main chart
    fig = visualizer.plot_stock_price(df, symbol, ['SMA_20', 'SMA_50', 'SMA_200'])

    # Summary table
    summary_data = {
        'Metric': ['Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'],
        'Value': [
            f"${df['Open'].iloc[-1]:.2f}",
            f"${df['High'].iloc[-1]:.2f}",
            f"${df['Low'].iloc[-1]:.2f}",
            f"${df['Close'].iloc[-1]:.2f}",
            f"{df['Volume'].iloc[-1]:,.0f}",
            f"${df['Close'].iloc[-1] * df['Volume'].iloc[-1]:,.0f}"
        ]
    }

    summary_table = dash_table.DataTable(
        data=pd.DataFrame(summary_data).to_dict('records'),
        columns=[{'name': i, 'id': i} for i in summary_data.keys()],
        style_cell={'textAlign': 'left'},
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    )

    return html.Div([
        html.H3(f"{symbol} - Overview", className="mb-4"),
        cards,
        dcc.Graph(figure=fig),
        html.Hr(),
        html.H4("Summary Statistics"),
        summary_table
    ])


def generate_predictions_content(df, symbol, model_selection):
    """Generate predictions tab content"""

    try:
        # Prepare data for prediction
        engineer = FeatureEngineer()
        result = engineer.engineer_all_features(df)

        # Split data
        split_idx = int(len(result['features']) * 0.8)
        X_train = result['features'][:split_idx]
        y_train = result['target'][:split_idx]
        X_test = result['features'][split_idx:]
        y_test = result['target'][split_idx:]

        # Initialize models
        trainer = StockPredictionModels(config.MODEL_CONFIG)

        predictions = {}

        # Train and predict based on selection
        if model_selection == 'all' or model_selection == 'linear':
            model = trainer.train_linear_regression(
                X_train.values, y_train.values,
                X_test.values, y_test.values
            )
            predictions['Linear Regression'] = trainer.predict('linear_regression', X_test.values)

        if model_selection == 'all' or model_selection == 'random_forest':
            model = trainer.train_random_forest(
                X_train.values, y_train.values,
                X_test.values, y_test.values
            )
            predictions['Random Forest'] = trainer.predict('random_forest', X_test.values)

        # Create predictions plot
        fig = visualizer.plot_predictions(y_test.values, predictions)

        # Performance metrics table
        metrics_data = []
        for model_name, preds in predictions.items():
            metrics = trainer.calculate_metrics(y_test.values, preds)
            metrics_data.append({
                'Model': model_name,
                'RMSE': f"{metrics['rmse']:.4f}",
                'MAE': f"{metrics['mae']:.4f}",
                'R²': f"{metrics['r2']:.4f}",
                'Direction Accuracy': f"{metrics['directional_accuracy']:.2%}"
            })

        metrics_table = dash_table.DataTable(
            data=metrics_data,
            columns=[{'name': i, 'id': i} for i in metrics_data[0].keys()] if metrics_data else [],
            style_cell={'textAlign': 'center'},
            style_data_conditional=[
                {
                    'if': {'column_id': 'Direction Accuracy'},
                    'color': 'green',
                    'fontWeight': 'bold'
                }
            ]
        )

        # Next day prediction
        latest_features = result['features'].iloc[-1:].values
        next_predictions = {}
        for model_name in trainer.models.keys():
            next_pred = trainer.predict(model_name, latest_features)[0]
            next_predictions[model_name] = next_pred

        pred_cards = dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H5(model_name),
                        html.H4(f"{pred:+.2%}",
                               className="text-success" if pred > 0 else "text-danger"),
                        html.P("Next Day Return", className="text-muted")
                    ])
                ])
            ], width=12//max(1, len(next_predictions)))
            for model_name, pred in next_predictions.items()
        ], className="mb-4")

        return html.Div([
            html.H3(f"{symbol} - Predictions", className="mb-4"),
            html.H5("Next Day Predictions"),
            pred_cards,
            html.Hr(),
            dcc.Graph(figure=fig),
            html.Hr(),
            html.H5("Model Performance Metrics"),
            metrics_table
        ])

    except Exception as e:
        logger.error(f"Error in predictions: {str(e)}")
        return html.Div([
            html.H3(f"{symbol} - Predictions", className="mb-4"),
            dbc.Alert(f"Error generating predictions: {str(e)}", color="danger")
        ])


def generate_technical_content(df, symbol):
    """Generate technical analysis content"""

    # Technical indicators plot
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=('Price & Moving Averages', 'MACD', 'RSI', 'Bollinger Bands'),
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )

    # Price and MAs
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='Price'
        ),
        row=1, col=1
    )

    for ma in ['SMA_20', 'SMA_50', 'SMA_200']:
        if ma in df.columns:
            fig.add_trace(
                go.Scatter(x=df.index, y=df[ma], name=ma, mode='lines'),
                row=1, col=1
            )

    # MACD
    if 'MACD' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue')),
            row=2, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['MACD_Signal'], name='Signal', line=dict(color='red')),
            row=2, col=1
        )
        fig.add_trace(
            go.Bar(x=df.index, y=df['MACD_Histogram'], name='Histogram'),
            row=2, col=1
        )

    # RSI
    if 'RSI_14' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['RSI_14'], name='RSI', line=dict(color='purple')),
            row=3, col=1
        )
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

    # Bollinger Bands
    if 'BB_Upper' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['BB_Upper'], name='BB Upper',
                      line=dict(color='gray', width=1)),
            row=4, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['BB_Lower'], name='BB Lower',
                      line=dict(color='gray', width=1), fill='tonexty'),
            row=4, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['Close'], name='Close',
                      line=dict(color='blue')),
            row=4, col=1
        )

    fig.update_layout(height=1000, title=f"{symbol} - Technical Analysis")
    fig.update_xaxes(rangeslider_visible=False)

    # Technical signals
    signals = []

    # Check for signals
    if 'RSI_14' in df.columns:
        rsi_val = df['RSI_14'].iloc[-1]
        if rsi_val > 70:
            signals.append(dbc.Alert("RSI indicates overbought conditions", color="warning"))
        elif rsi_val < 30:
            signals.append(dbc.Alert("RSI indicates oversold conditions", color="info"))

    if 'MACD' in df.columns and 'MACD_Signal' in df.columns:
        if df['MACD'].iloc[-1] > df['MACD_Signal'].iloc[-1]:
            signals.append(dbc.Alert("MACD shows bullish crossover", color="success"))
        else:
            signals.append(dbc.Alert("MACD shows bearish crossover", color="danger"))

    return html.Div([
        html.H3(f"{symbol} - Technical Analysis", className="mb-4"),
        html.Div(signals),
        dcc.Graph(figure=fig)
    ])


def generate_performance_content(df, model_selection):
    """Generate model performance content"""

    # Create sample comparison data
    comparison_data = pd.DataFrame({
        'Model': ['Linear Regression', 'Random Forest', 'LSTM', 'Ensemble'],
        'Train_RMSE': [0.012, 0.008, 0.010, 0.009],
        'Val_RMSE': [0.015, 0.011, 0.013, 0.012],
        'Train_R2': [0.85, 0.92, 0.88, 0.90],
        'Val_R2': [0.82, 0.89, 0.86, 0.88],
        'Train_DirectionalAcc': [0.62, 0.68, 0.65, 0.67],
        'Val_DirectionalAcc': [0.58, 0.64, 0.61, 0.63],
        'Overfit_Score': [0.003, 0.003, 0.003, 0.003]
    })

    # Performance comparison chart
    fig = visualizer.plot_model_comparison(comparison_data)

    # Feature importance (mock data)
    importance_data = pd.DataFrame({
        'feature': ['RSI_14', 'MACD', 'SMA_20', 'Volume', 'BB_Position',
                    'Volatility_20', 'Close_Lag_1', 'Returns_Lag_1'],
        'importance': [0.15, 0.12, 0.10, 0.09, 0.08, 0.07, 0.06, 0.05]
    })

    fig_importance = visualizer.plot_feature_importance(importance_data, top_n=8)

    # Cross-validation results
    cv_results = dbc.Card([
        dbc.CardHeader("Cross-Validation Results"),
        dbc.CardBody([
            html.P("5-Fold Time Series Cross-Validation"),
            html.Ul([
                html.Li(f"Average RMSE: 0.0125 (±0.002)"),
                html.Li(f"Average R²: 0.85 (±0.03)"),
                html.Li(f"Average MAE: 0.009 (±0.001)")
            ])
        ])
    ])

    return html.Div([
        html.H3("Model Performance Analysis", className="mb-4"),
        dcc.Graph(figure=fig),
        html.Hr(),
        dbc.Row([
            dbc.Col([
                html.H5("Feature Importance"),
                dcc.Graph(figure=fig_importance)
            ], width=8),
            dbc.Col([
                cv_results
            ], width=4)
        ])
    ])


def generate_backtesting_content(df, symbol):
    """Generate backtesting content"""

    # Mock backtesting results
    dates = df.index[-100:]
    initial_capital = 100000
    returns = np.random.randn(100) * 0.02
    portfolio_value = initial_capital * (1 + returns).cumprod()
    benchmark_value = initial_capital * (1 + np.random.randn(100) * 0.015).cumprod()

    # Calculate drawdown
    running_max = np.maximum.accumulate(portfolio_value)
    drawdown = (portfolio_value - running_max) / running_max * 100

    backtest_data = {
        'dates': dates,
        'portfolio_value': portfolio_value,
        'benchmark_value': benchmark_value,
        'returns': returns * 100,
        'drawdown': drawdown
    }

    fig = visualizer.plot_backtesting_results(backtest_data)

    # Performance metrics
    total_return = (portfolio_value[-1] - initial_capital) / initial_capital * 100
    sharpe_ratio = returns.mean() / returns.std() * np.sqrt(252)
    max_dd = drawdown.min()
    win_rate = np.mean(returns > 0)

    metrics_cards = dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Total Return"),
                    html.H3(f"{total_return:+.2f}%",
                           className="text-success" if total_return > 0 else "text-danger")
                ])
            ])
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Sharpe Ratio"),
                    html.H3(f"{sharpe_ratio:.2f}")
                ])
            ])
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Max Drawdown"),
                    html.H3(f"{max_dd:.2f}%", className="text-danger")
                ])
            ])
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Win Rate"),
                    html.H3(f"{win_rate:.1%}")
                ])
            ])
        ], width=3)
    ], className="mb-4")

    return html.Div([
        html.H3(f"{symbol} - Backtesting Results", className="mb-4"),
        metrics_cards,
        dcc.Graph(figure=fig)
    ])


def generate_settings_content():
    """Generate settings tab content"""

    return html.Div([
        html.H3("Settings", className="mb-4"),
        dbc.Card([
            dbc.CardBody([
                html.H5("Model Configuration"),
                html.Hr(),
                dbc.Row([
                    dbc.Col([
                        html.Label("LSTM Epochs"),
                        dbc.Input(type="number", value=100, min=10, max=500)
                    ], width=4),
                    dbc.Col([
                        html.Label("Random Forest Trees"),
                        dbc.Input(type="number", value=100, min=10, max=1000)
                    ], width=4),
                    dbc.Col([
                        html.Label("Sequence Length"),
                        dbc.Input(type="number", value=60, min=10, max=200)
                    ], width=4)
                ]),
                html.Hr(),
                html.H5("Data Settings"),
                dbc.Row([
                    dbc.Col([
                        html.Label("Train/Test Split"),
                        dcc.Slider(min=0.5, max=0.9, value=0.8, step=0.05,
                                  marks={i/10: f"{i*10}%" for i in range(5, 10)})
                    ], width=12)
                ]),
                html.Hr(),
                dbc.Button("Save Settings", color="primary", className="mt-3")
            ])
        ])
    ])


# Callbacks
@app.callback(
    Output('stored-data', 'children'),
    [Input('update-button', 'n_clicks')],
    [State('stock-symbol', 'value'),
     State('date-range', 'start_date'),
     State('date-range', 'end_date')]
)
def fetch_and_process_data(n_clicks, symbol, start_date, end_date):
    """Fetch and process stock data"""
    if not n_clicks:
        return json.dumps({})

    try:
        # Fetch data
        collector = StockDataCollector(symbol, start_date, end_date)
        data = collector.prepare_features()

        # Convert to JSON-serializable format
        data_dict = {
            'symbol': symbol,
            'data': data.to_json(date_format='iso'),
            'last_update': datetime.now().isoformat()
        }

        return json.dumps(data_dict)

    except Exception as e:
        logger.error(f"Error fetching data: {str(e)}")
        return json.dumps({'error': str(e)})


@app.callback(
    Output('tab-content', 'children'),
    [Input('main-tabs', 'value'),
     Input('stored-data', 'children'),
     Input('model-selection', 'value')]
)
def update_tab_content(active_tab, stored_data_json, model_selection):
    """Update content based on selected tab"""

    # Parse stored data
    try:
        stored_data = json.loads(stored_data_json) if stored_data_json else {}
    except:
        stored_data = {}

    if 'error' in stored_data:
        return dbc.Alert(f"Error: {stored_data['error']}", color="danger")

    if not stored_data or 'data' not in stored_data:
        return dbc.Alert("Please select a stock and click Update to load data.", color="info")

    # Load data
    df = pd.read_json(stored_data['data'])
    symbol = stored_data.get('symbol', 'Stock')

    # Generate content based on active tab
    if active_tab == 'overview':
        return generate_overview_content(df, symbol)

    elif active_tab == 'predictions':
        return generate_predictions_content(df, symbol, model_selection)

    elif active_tab == 'technical':
        return generate_technical_content(df, symbol)

    elif active_tab == 'performance':
        return generate_performance_content(df, model_selection)

    elif active_tab == 'backtesting':
        return generate_backtesting_content(df, symbol)

    elif active_tab == 'settings':
        return generate_settings_content()

    return html.Div()


@app.callback(
    Output('status-bar', 'children'),
    [Input('stored-data', 'children')]
)
def update_status(stored_data_json):
    """Update status bar"""
    try:
        stored_data = json.loads(stored_data_json) if stored_data_json else {}
        if 'last_update' in stored_data:
            return f"Last updated: {stored_data['last_update']}"
    except:
        pass
    return "Ready"


# Run the app
if __name__ == '__main__':
    app.run(
        debug=config.DASHBOARD_CONFIG['debug'],
        host=config.DASHBOARD_CONFIG['host'],
        port=config.DASHBOARD_CONFIG['port']
    )